# Data Modelling

In [93]:
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from helpers.helper_functions import translate_to_file_string
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics

inputFile = translate_to_file_string("./data/Data_Preparation_Result.csv")

def prettyPrint(dm, collArray) :
    rows = dm.toArray().tolist()
    dfDM = spark.createDataFrame(rows,collArray)
    newDf = dfDM.toPandas()
    from IPython.display import display, HTML
    return HTML(newDf.to_html(index=False))

## Create Spark Session

In [94]:
#create a SparkSession
spark = (SparkSession
       .builder
       .appName("DataModelling")
       .getOrCreate())
# create a DataFrame using an ifered Schema 
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ";") \
       .csv(inputFile)   
print(df.printSchema())

root
 |-- Bundesland: string (nullable = true)
 |-- BundeslandIndex: integer (nullable = true)
 |-- Landkreis: string (nullable = true)
 |-- LandkreisIndex: integer (nullable = true)
 |-- Altersgruppe: string (nullable = true)
 |-- AltersgruppeIndex: double (nullable = true)
 |-- Geschlecht: string (nullable = true)
 |-- GeschlechtIndex: double (nullable = true)
 |-- FallStatus: string (nullable = true)
 |-- FallStatusIndex: double (nullable = true)
 |-- Falldatum: string (nullable = true)

None


## Vorbereitung der Daten

### Filtern der Datensätze
Für das Training dieses Modells ist es sinnvoll nur die Fälle zu betrachten, bei den der Ausgang der Corona-Erkrankung bereits bekannt ist ("GENESEN" oder "GESTORBEN"). Daher werden die Fälle mit noch erkrankten Personen herausgefiltert. Ebenfalls muss der FallStatusIndex neu vergeben werden, damit dieses Feature nur noch die Werte 0 oder 1 enthält.

In [95]:
dfNeu = df.filter(df.FallStatus != "NICHTEINGETRETEN").drop("FallStatusIndex")

### FallStatusIndex

In [96]:
# Wollten eigentlich den Indexer mitgeben in die Pipeline. Dies führt aber zum Fehler, dass er das Label nicht kennt.
indexer = StringIndexer(inputCol="FallStatus", outputCol="FallStatusIndex")
dfReindexed = indexer.fit(dfNeu).transform(dfNeu)

### Ziehen eines Samples
Da der Datensatz sehr groß ist,kann es evt. notwendig sein, nur mit einem kleineren Umfang zu trainieren. Mit Fraction kann an dieser Stelle der Umfang der Stichprobe angepasst werden.

In [97]:
dfsample = dfReindexed.sample(withReplacement=False, fraction=1.0, seed=12334556)

### Undersampling
Ähnlich dem Fraud-Detection-Beispiel von Tara Boyle (2019) ist die Klasse der an Corona-Verstorbenen im vorliegenden Datensatz unterrepresentiert, weshalb man an dieser Stelle von einer Data Imbalance spricht. Dies sieht man wenn man die Anzahl der Todesfälle mit den Anzahl der Genesenen vergleicht.

In [98]:
# Vergleich der Fallzahlen
dfsample.groupBy("FallStatus").count().show()

+----------+-------+
|FallStatus|  count|
+----------+-------+
|   GENESEN|3471830|
| GESTORBEN|  88350|
+----------+-------+



Die meisten Machine Learning Algorithmen arbeiten am Besten wenn die Nummer der Samples in allen Klassen ungefähr die selbe größe haben. Dies konnte auch im Zuge dieser Arbeit bei den unterschiedlichen Regressions-Modellen festgestellt werden. Da die einzelnen Modelle versuchen den Fehler zu reduzieren, haben alle Modelle am Ende für einen Datensatz nur die Klasse Genesen geliefert, da hier die Wahrscheinlichkeit am größten war korrekt zu liegen. 
Um diese Problem zu lösen gibt es zwei Möglichkeiten: Under- und Oversampling. Beides fällt unter den Begriff Resampling
Beim Undersampling werden aus der Klasse mit den meisten Instanzen, Datensätze gelöscht, wohingegen beim Oversampling, der Klasse mit den wenigsten Isntanzen, neue Werte hinzugefügt werden. (Will Badr 2019; Tara Boyle 2019)
Da in diesem Fall ausreichend Datensätze vorhanden sind, bietet sich Ersteres an.

In [99]:
# Ermittlung der Anzahl dr Verstorbenen
dfGestorben = dfsample.filter(dfsample.FallStatus == "GESTORBEN")
anzahlGestorben = dfGestorben.count()
print("Anzahl Gestorben : %s" % anzahlGestorben)

Anzahl Gestorben : 88350


In [100]:
# Ermittlung des Verhätlnisses von Verstorben und Gensen
dfGenesen = dfsample.filter(dfsample.FallStatus == "GENESEN")
anzahlGenesen = dfGenesen.count()
print("Anzahl Genesen : %s" % anzahlGenesen)

ratio = anzahlGestorben / anzahlGenesen
print("Verhältnis : %s" % ratio)



Anzahl Genesen : 3471830
Verhältnis : 0.02544767456931935


In [101]:
# Ziehen eines Samples mit der näherungsweise selben Anzahl wie Verstorbene
dfGenesenSample = dfGenesen.sample(fraction=ratio, seed=12345)

In [102]:
dfGesamtSample = dfGestorben.union(dfGenesenSample)
# Kontrolle
dfGesamtSample.groupBy("FallStatus").count().show()

+----------+-----+
|FallStatus|count|
+----------+-----+
|   GENESEN|88520|
| GESTORBEN|88350|
+----------+-----+



### Splitten in Trainings und Testdaten

In [103]:
splits = dfGesamtSample.randomSplit([0.8, 0.2], 345678)
trainingData = splits[0]
testData = splits[1]

### Aufbau des Feature-Vectors

In [104]:
assembler =  VectorAssembler(outputCol="features", inputCols=["GeschlechtIndex","AltersgruppeIndex", "LandkreisIndex","BundeslandIndex"])

## Modellierung
### Logistic Regression
Logistic Regression ist eine sehr beliebte Methode um Kategorien vorherzusagen. Für Binäre Werte ist eine Binominal Logistic Regresision anzuwenden. (Apache Spark 2021a)

In [105]:
lr = LogisticRegression(maxIter=10, featuresCol="features", family="binomial", labelCol="FallStatusIndex")


### Pipeline

In [106]:
pipeline = Pipeline(stages=[assembler,lr])

### Evaluator
Für die spätere Cross-Validaton wird ein Evaluator benötigt. Letzterer ist zu wählen, abhängig von dem jeweilligen Modell und Anwendungsfall. (Apache Spark 2020a) In diesem Fall wird ein BinaryClassificationEvaluator angewendet. Dieser eignet sich besonders für binäre Werte. (Apache Spark 2021a) Da Geschlecht, in diesem Fall, der FallStatus 0 oder 1 annehmen kann, bietet er sich hier besonders an.

In [107]:
# Definition des Evaluators
evaluator= BinaryClassificationEvaluator(labelCol="FallStatusIndex",rawPredictionCol="prediction", metricName="areaUnderPR")

### Parametertuning
Eine wichtige Aufgabe beim Machine Learning ist die Auswahl des geeigneten Modells bzw. die passenden Paramter für ein Modell herauszufinden. Letzteres wird auch Parametertuning genannt. Die in Pyspark enthaltene MLLib bietet speziell hierfür ein entsprechende Tooling. Und zwar kann ein CrossValidator bzw. ein TrainValidationSplit verwendet werden. Voraussetzung sind ein Estimator (ein Modell oder eine Pipeline), ein Paramter-Grid und eine Evaluator. Dies ist auch im Zusammenhang mit dem Thema Cross-Validation zu sehen. (Apache Spark 2020a)

In [108]:
paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.3,0.01]) \
    .addGrid(lr.elasticNetParam, [0.8])\
    .addGrid(evaluator.metricName, ["areaUnderPR", "areaUnderROC"])\
    .build()

### Cross-Validation

In [109]:
# Definition des Cross-Validators 
# num-Folds gibt an in wie viele Datensatz-Paare die Datensätze aufgeteilt werden.
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2,
                          parallelism=2)

#### Training

In [110]:
# Anpassung des Modells und Auswahl der besten Parameter
cvModel = crossval.fit(trainingData)

In [111]:
# Ermitteln der Paramter
print(cvModel.explainParams())

estimator: estimator to be cross-validated (current: Pipeline_e5c23f810a28)
estimatorParamMaps: estimator param maps (current: [{Param(parent='LogisticRegression_403c859db9cc', name='regParam', doc='regularization parameter (>= 0).'): 0.3, Param(parent='LogisticRegression_403c859db9cc', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.8, Param(parent='BinaryClassificationEvaluator_5fa397b28bc5', name='metricName', doc='metric name in evaluation (areaUnderROC|areaUnderPR)'): 'areaUnderPR'}, {Param(parent='LogisticRegression_403c859db9cc', name='regParam', doc='regularization parameter (>= 0).'): 0.3, Param(parent='LogisticRegression_403c859db9cc', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.8, Param(parent='BinaryClassificationEvaluator_5fa397b28bc5', na

### Testen des Modells

In [112]:
predictions = cvModel.transform(testData)
predictions.show()

+-----------------+---------------+----------------+--------------+------------+-----------------+----------+---------------+----------+----------+---------------+--------------------+--------------------+--------------------+----------+
|       Bundesland|BundeslandIndex|       Landkreis|LandkreisIndex|Altersgruppe|AltersgruppeIndex|Geschlecht|GeschlechtIndex|FallStatus| Falldatum|FallStatusIndex|            features|       rawPrediction|         probability|prediction|
+-----------------+---------------+----------------+--------------+------------+-----------------+----------+---------------+----------+----------+---------------+--------------------+--------------------+--------------------+----------+
|Baden-Württemberg|              8|     LK Biberach|          8426|     A60-A79|              2.0|         M|            1.0| GESTORBEN|2020-04-14|            1.0|[1.0,2.0,8426.0,8.0]|[0.01184058670608...|[0.50296011209275...|       0.0|
|Baden-Württemberg|              8|     LK Biber

In [113]:
# Kontrolle der Predictions
predictions.groupBy("FallStatusIndex", "prediction").count().show()

+---------------+----------+-----+
|FallStatusIndex|prediction|count|
+---------------+----------+-----+
|            1.0|       1.0|11852|
|            0.0|       1.0| 2850|
|            1.0|       0.0| 5987|
|            0.0|       0.0|14718|
+---------------+----------+-----+



## Modell - Evaluation

Area Under PR / Area under ROC

In [114]:
accuracy = evaluator.evaluate(predictions)
print("Test Error",(1.0 - accuracy))

Test Error 0.24458275341925528


### BinaryClassificationMetrics

Bei dem untersuchten Label (FallStatus mit den Ausprägungen Verstorben und Genesen) handelt es sich um ein einen BinaryClasificator. Er kann die Werte 0 und 1 annehmen. Für die Modellevaluation sind daher die BinaryClassificationMetrics zu verwenden. (Apache Spark 2021i)

In [115]:
predictionAndLabels = predictions.select("prediction", "FallStatusIndex").rdd.map(lambda p: [p[0], p[1]]) # Map to RDD prediction|label

In [116]:
# Instanzieire das BinaryClassificationMetrics-Objekt
metrics = BinaryClassificationMetrics(predictionAndLabels)

# Fläche unter der Precision-recall Curve

print("Area under PR = %s" % metrics.areaUnderPR)

# Fläche unter der ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)

Area under PR = 0.7554172465807447
Area under ROC = 0.7510801206266003


### Multiclass classification Metrics
In den meißten Fällen können auch Multiclass Classification Metrics bei Binary Classifaction Problemen angewandt werden.

In [117]:
predictionAndLabels = predictions.select("prediction", "FallStatusIndex").rdd.map(lambda p: (p[0], p[1])) # Map to RDD prediction|label
# Instantiate metrics object
mcMetrics = MulticlassMetrics(predictionAndLabels)

In [118]:
prettyPrint(mcMetrics.confusionMatrix(),["positiv", "negativ"])

positiv,negativ
14718.0,2850.0
5987.0,11852.0


In [119]:
# Overall statistics
precision = mcMetrics.precision(1.0)
recall = mcMetrics.recall(1.0)
f1Score = mcMetrics.fMeasure(1.0)
print("Summary Stats")
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1 Score = %s" % f1Score)


labels = predictions.select("FallStatusIndex").rdd.map(lambda lp: lp.FallStatusIndex).distinct().collect()
for label in sorted(labels):
    print("Class %s precision = %s" % (label, mcMetrics.precision(label)))
    print("Class %s recall = %s" % (label, mcMetrics.recall(label)))
    print("Class %s F1 Measure = %s" % (label, mcMetrics.fMeasure(label, beta=1.0)))

# Weighted stats
print("Weighted recall = %s" % mcMetrics.weightedRecall)
print("Weighted precision = %s" % mcMetrics.weightedPrecision)
print("Weighted F(1) Score = %s" % mcMetrics.weightedFMeasure())
print("Weighted F(0.5) Score = %s" % mcMetrics.weightedFMeasure(beta=0.5))
print("Weighted false positive rate = %s" % mcMetrics.weightedFalsePositiveRate)

Summary Stats
Precision = 0.8061488232893483
Recall = 0.6643870172094848
F1 Score = 0.7284348975139056
Class 0.0 precision = 0.7108427915962328
Class 0.0 recall = 0.8377732240437158
Class 0.0 F1 Measure = 0.7691061583884201
Class 1.0 precision = 0.8061488232893483
Class 1.0 recall = 0.6643870172094848
Class 1.0 F1 Measure = 0.7284348975139056
Weighted recall = 0.7504165842912418
Weighted precision = 0.758860536657195
Weighted F(1) Score = 0.7486148820097248
Weighted F(0.5) Score = 0.7532587545987341
Weighted false positive rate = 0.24825634303804112
